In [2]:
import pyterrier as pt
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, lil_matrix
from sklearn.decomposition import TruncatedSVD

# Listing all files in the collection
files = pt.io.find_files(r'C:\Users\Dell\Desktop\IR Lab\Lab4\lab4\data\en_BDNews24\en_BDNews24\5')
import time

In [3]:
print(f"Num files: {len(files)}")

Num files: 131


In [101]:
# Indexing and Retrieval Experiments.
# Skip this if using a prebuilt index

In [ ]:
# Indexing the collection

indexer = pt.TRECCollectionIndexer(r"C:\Users\Dell\Desktop\IR Lab\Lab4\lab4\Index", 
                                   meta = {'docno' : 75},
                                   verbose=True, 
                                   blocks=False)
indexref = indexer.index(files)


  1%|          | 996/89286 [00:30<22:17, 65.99files/s]

In [7]:
# load the index, print the statistics
index = pt.IndexFactory.of(r"C:\Users\Dell\Desktop\IR Lab\Lab4\lab4\fire_2012_english_index\fire_2012\data.properties")
print(index.getCollectionStatistics().toString())

Number of documents: 392577
Number of terms: 393188
Number of postings: 61291043
Number of fields: 0
Number of tokens: 95881418
Field names: []
Positions:   false



In [5]:
# Loading all queries in the collection
topics = pt.io.read_topics(r'C:\Users\Dell\Desktop\IR Lab\Lab4\lab4\english-queries.xml')

12:10:32.906 [main] WARN org.terrier.applications.batchquerying.TRECQuery -- trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


Java started (triggered by _read_topics_trec) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


In [6]:
# Loading the relevance Judgments
qrels = pt.io.read_qrels(r'C:\Users\Dell\Desktop\IR Lab\Lab4\lab4\en.qrels.176-225.2012-v2.txt')

In [36]:
# Defining which retrieval models to use

tf_idf = pt.terrier.Retriever(index, wmodel="TF_IDF")
bm25 = pt.terrier.Retriever(index, wmodel="BM25")

# Using bm25 + query expansion using KL divergence
bm25_qe = pt.terrier.Retriever(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "KL"})

In [37]:
# Retrieving results for specific query vs entire query set

single_results = bm25.transform("ysr reddy death")
batch_results = bm25.transform(topics)

/home/parth/Installations/anaconda3/envs/nlp/lib/python3.10/site-packages/pyterrier/utils.py:207: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return fn(*args, **kwargs)


In [38]:
single_results

,qid,docid,docno,rank,score,query
0,1,214902,1090905_frontpage_story_11454541.utf8,0,38.632609,ysr reddy death
1,1,219778,1090904_nation_story_11449653.utf8,1,38.443461,ysr reddy death
2,1,219853,1090908_nation_story_11465393.utf8,2,37.748173,ysr reddy death
3,1,253249,1101130_nation_story_13238955.utf8,3,37.515744,ysr reddy death
4,1,253283,1101202_nation_story_13248378.utf8,4,36.221675,ysr reddy death
...,...,...,...,...,...,...
995,1,153826,1040511_nation_story_3231721.utf8,995,10.119484,ysr reddy death
996,1,123223,1030622_nation_story_2092320.utf8,996,10.109404,ysr reddy death
997,1,124759,1030920_nation_story_2384499.utf8,997,10.109404,ysr reddy death
998,1,382492,1070827_nation_story_8241642.utf8,998,10.109404,ysr reddy death


In [39]:
batch_results[batch_results.qid == '176']

,qid,docid,docno,rank,score,query
0,176,214902,1090905_frontpage_story_11454541.utf8,0,38.632609,ysr reddy death
1,176,219778,1090904_nation_story_11449653.utf8,1,38.443461,ysr reddy death
2,176,219853,1090908_nation_story_11465393.utf8,2,37.748173,ysr reddy death
3,176,253249,1101130_nation_story_13238955.utf8,3,37.515744,ysr reddy death
4,176,253283,1101202_nation_story_13248378.utf8,4,36.221675,ysr reddy death
...,...,...,...,...,...,...
995,176,153826,1040511_nation_story_3231721.utf8,995,10.119484,ysr reddy death
996,176,123223,1030622_nation_story_2092320.utf8,996,10.109404,ysr reddy death
997,176,124759,1030920_nation_story_2384499.utf8,997,10.109404,ysr reddy death
998,176,382492,1070827_nation_story_8241642.utf8,998,10.109404,ysr reddy death


In [40]:
# Running the experiments and computing the following metrics
# 1. MAP
# 2. P@5
# 3. P@10
# 4. P@100

results_df = pt.Experiment([tf_idf, bm25, bm25_qe], 
                           topics, 
                           qrels, 
                           eval_metrics=["map", "P_5", "P_10", "P_100"], 
                           perquery=False,
                           names=["TF_IDF", "BM25", "BM25_QE"])

  0%|                                             | 0/392577 [14:52<?, ?files/s]


In [41]:
results_df

,name,map,P_5,P_10,P_100
0,TF_IDF,0.354835,0.564,0.522,0.2884
1,BM25,0.351517,0.568,0.510,0.2898
2,BM25_QE,0.394997,0.584,0.552,0.3182


In [103]:
# Start here if using preexisting index

In [151]:
# Load preexisting index

index = pt.IndexFactory.of("/path/to/lab4/fire_2012_BDNews/")
print(index.getCollectionStatistics())

Number of documents: 392577
Number of terms: 393188
Number of postings: 61291043
Number of fields: 0
Number of tokens: 95881418
Field names: []
Positions:   false



In [152]:
# Accessing the lexicon in the index
lexicon = index.getLexicon()
total_terms = 393188
total_docs = 392577 # From above

In [123]:
# Print first 10 terms in the lexicon 
# with their ids, tf and idf values and pointers to postings

count = 0
for entry in lexicon:
    count += 1
    term_text = entry.getKey()
    term_id = entry.getValue()
    print(term_text, term_id)
    if count == 10:
        break

0 term0 Nt=36041 TF=425928 maxTF=2147483647 @{0 0 0}
00 term3213 Nt=5239 TF=26675 maxTF=2147483647 @{0 74173 7}
000 term196 Nt=66067 TF=115581 maxTF=2147483647 @{0 83496 3}
0001 term90971 Nt=81 TF=103 maxTF=2147483647 @{0 133598 5}
0002 term155078 Nt=89 TF=101 maxTF=2147483647 @{0 133813 3}
0003 term158704 Nt=98 TF=129 maxTF=2147483647 @{0 134037 7}
0004 term155077 Nt=114 TF=164 maxTF=2147483647 @{0 134279 0}
0005 term91643 Nt=11 TF=11 maxTF=2147483647 @{0 134557 2}
0006 term244548 Nt=4 TF=4 maxTF=2147483647 @{0 134593 4}
0007 term209011 Nt=4 TF=4 maxTF=2147483647 @{0 134607 4}


In [238]:
di = index.getDirectIndex() #The actual posting lists
doi = index.getDocumentIndex() # Document information
docid = 12

# Print term id and frequency
print(f"Document Length: {posting.getDocumentLength()}")

for posting in di.getPostings(doi.getDocumentEntry(docid)):
    print(posting.getId(), posting.getFrequency())


Document Length: 108
5 1
10 1
31 2
46 1
52 1
63 1
71 1
75 1
82 3
95 1
103 1
104 1
119 1
132 1
144 5
168 2
170 3
181 1
191 1
192 1
195 2
196 1
211 3
233 1
240 1
241 1
249 1
255 3
263 1
271 2
278 1
279 2
281 1
282 2
284 1
291 1
308 2
309 1
320 1
321 2
325 1
331 1
344 1
345 1
360 1
381 1
383 1
398 1
401 1
412 2
428 1
435 1
444 3
458 2
467 4
477 1
491 2
496 2
503 2
520 8
552 1
573 2
618 1
643 2
672 1
682 1
701 4
736 9
748 2
766 1
772 1
780 1
786 2
835 2
836 1
857 1
858 3
863 4
868 1
894 3
901 2
930 2
933 1
999 1
1015 1
1036 1
1060 1
1070 2
1090 1
1132 1
1134 1
1147 1
1151 1
1157 1
1168 1
1178 2
1187 1
1228 1
1232 1
1237 1
1238 1
1240 1
1253 1
1256 1
1269 2
1285 1
1293 1
1294 1
1300 2
1335 1
1343 1
1377 1
1378 1
1397 1
1408 2
1412 2
1414 1
1429 2
1434 1
1440 1
1479 1
1547 1
1556 1
1583 2
1584 1
1585 1
1586 1
1587 1
1588 1
1589 4
1590 1
1591 2
1592 1
1593 1
1594 1
1595 2
1596 1
1597 1
1598 11
1599 1
1600 1
1601 1
1602 1
1603 1
1604 6
1605 1
1606 8
1607 1
1608 1
1609 2
1610 1
1611 1
1612 1
16

In [144]:
# Failed attempt at allocting enough memory.
# For this smallish dataset, the full matrix is 1.2TB!

td_matrix =  np.zeros((total_docs, total_terms))



MemoryError: Unable to allocate 1.12 TiB for an array with shape (392577, 393188) and data type float64

In [159]:
# Lets try using sparse matrix

sparse_matrix = csr_matrix((total_docs, total_terms))

# FIll up the csr matrix with TD values

start = time.time()
for docid in range(total_docs):
    if (docid % 10 == 0) and (docid != 0):
        print(f"Processed {docid} in {(time.time()-start)/60} mins")

    for posting in di.getPostings(doi.getDocumentEntry(docid)):
        termid = posting.getId()
        sparse_matrix[docid, termid] = posting.getFrequency()
    if (docid % 100 == 0) and (docid !=0):
        print("Enough to demonstrate the slowness")
        break

Processed 10 in 0.09291206995646159 mins
Processed 20 in 0.1537517468134562 mins
Processed 30 in 0.22062720060348512 mins
Processed 40 in 0.2731042464574178 mins
Processed 50 in 0.34677772521972655 mins
Processed 60 in 0.42887327671051023 mins
Processed 70 in 0.5074725270271301 mins
Processed 80 in 0.5763836741447449 mins
Processed 90 in 0.6667742808659871 mins
Processed 100 in 0.7696935296058655 mins
Enough to demonstrate the slowness


In [160]:
# Initialize a list of list matrix
# This is the most efficient implementation when the size of your sparse matrix keeps changing

td_matrix =  lil_matrix((392577, 393188))

# Fill up the lil_matrix with TD values

start = time.time()
for docid in range(total_docs):
    if docid % 10000 == 0:
        print(f"Processed {docid} in {(time.time()-start)/60} mins")

    for posting in di.getPostings(doi.getDocumentEntry(docid)):
        termid = posting.getId()
        td_matrix[docid, termid] = posting.getFrequency()

Processed 0 in 2.8888384501139324e-06 mins
Processed 10000 in 0.20643712679545084 mins
Processed 20000 in 0.3622871955235799 mins
Processed 30000 in 0.49738730986913043 mins
Processed 40000 in 0.6417509118715922 mins
Processed 50000 in 0.8323306282361348 mins
Processed 60000 in 1.0655373613039651 mins
Processed 70000 in 1.2449093421300252 mins
Processed 80000 in 1.4480005979537964 mins
Processed 90000 in 1.675760285059611 mins
Processed 100000 in 1.9203487237294514 mins
Processed 110000 in 2.1255168000857037 mins
Processed 120000 in 2.3662728389104206 mins
Processed 130000 in 2.6219697833061217 mins
Processed 140000 in 2.8151995102564493 mins
Processed 150000 in 3.0381730715433757 mins
Processed 160000 in 3.3228824377059936 mins
Processed 170000 in 3.5408313870429993 mins
Processed 180000 in 3.7896983941396076 mins
Processed 190000 in 4.044611291090647 mins
Processed 200000 in 4.278400671482086 mins
Processed 210000 in 4.53971213499705 mins
Processed 220000 in 4.769796423117319 mins
Pr

In [167]:
sparse_matrix = csr_matrix(td_matrix)

In [242]:
# Print terms and frequencies for a document
doc_id = 12

# Row for a specific doc_id converted to dense matrix and iterated over it
# Gives term id and frequency
for xid in np.where(sparse_matrix[doc_id,:].todense().A1)[0]:
    lee = lex.getLexiconEntry(int(xid))
    print("id: %d, term: %s, Frequency %d" % (int(xid), lee.getKey(), sparse_matrix[doc_id,xid]))

id: 5, term: bdnews24, Frequency 1
id: 10, term: win, Frequency 1
id: 31, term: try, Frequency 2
id: 46, term: 1, Frequency 1
id: 52, term: taken, Frequency 1
id: 63, term: com, Frequency 1
id: 71, term: reuter, Frequency 1
id: 75, term: world, Frequency 1
id: 82, term: court, Frequency 3
id: 95, term: deni, Frequency 1
id: 103, term: public, Frequency 1
id: 104, term: develop, Frequency 1
id: 119, term: health, Frequency 1
id: 132, term: prime, Frequency 1
id: 144, term: build, Frequency 5
id: 168, term: onli, Frequency 2
id: 170, term: famili, Frequency 3
id: 181, term: servic, Frequency 1
id: 191, term: minist, Frequency 1
id: 192, term: intern, Frequency 1
id: 195, term: wai, Frequency 2
id: 196, term: 000, Frequency 1
id: 211, term: sai, Frequency 3
id: 233, term: report, Frequency 1
id: 240, term: dozen, Frequency 1
id: 241, term: activ, Frequency 1
id: 249, term: found, Frequency 1
id: 255, term: right, Frequency 3
id: 263, term: jun, Frequency 1
id: 271, term: call, Frequency 2

In [253]:
# Another failed attempt at allocating memory
# Cannot compute SVD directly eve for 100 documents and all terms
dense_td_matrix_small = sparse_matrix[0:100, :].todense()

U, Sigma, VT = np.linalg.svd(dense_td_matrix_small)

MemoryError: Unable to allocate 1.12 TiB for an array with shape (393188, 393188) and data type float64

In [257]:
# Method one of computing SVD

start = time.time()

svd = TruncatedSVD(n_components=200)  # Reduce the matrix to 200 components
sparse_svd_result = svd.fit_transform(dense_td_matrix_small)

print(f"completed in {time.time() - start} seconds")

/home/parth/Installations/anaconda3/envs/nlp/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


completed in 12.238288640975952 seconds


In [262]:
# Alternate approach is applying SVD on sparse matrix directly

from scipy.sparse.linalg import svds

small_sparse_matrix = sparse_matrix[0:100, :]

start = time.time()
# Perform sparse SVD

# Notice how svds doesnt allow larger value of k
# Try changing K to something > 99
U, Sigma, VT = svds(small_sparse_matrix, k=99)  
print(f"Processed in {(time.time() - start)/60} minutes")

Processed in 0.04333812395731608 minutes


In [276]:
# Now SVD for full matrix

start = time.time()
# Perform sparse SVD

# Play around with different values of K

U, Sigma, VT = svds(sparse_matrix.transpose(), k=200)  # Transposing because we made an error while creating the matrix
print(f"Processed in {(time.time() - start)/60} minutes")

Processed in 6.037864538033803 minutes


In [277]:
# LArgest singular values

print(Sigma)

[  414.48251234   415.33748686   416.11426393   416.81099118
   417.47123147   418.00202349   418.54986896   418.60315885
   420.63616698   421.30049844   422.863892     423.23750769
   424.05062742   424.83034401   426.45304841   427.00191228
   427.68213523   428.67319982   429.44098869   431.90942353
   432.32773671   433.79723359   434.62241568   435.21396593
   436.58720691   437.74315283   438.41921394   439.35191773
   440.13494548   442.49408379   443.50570587   444.82013967
   445.38752193   446.32814713   449.22302986   450.6576348
   451.83367089   452.34870945   453.51997365   453.76314903
   455.92723764   456.34993624   458.15219101   458.37999847
   461.51809388   461.8130129    462.00032919   462.91012965
   464.02824203   464.38600989   467.01642887   469.56396166
   470.445459     471.59423256   472.3766871    473.66939934
   474.32482973   475.36186692   476.78694369   478.35453256
   479.60777561   480.81227479   484.2337019    484.73374132
   485.55426547   486.898

In [279]:
# Find id for a particular word

for x in range(total_terms):
    if lex.getLexiconEntry(int(x)).getKey() == 'barbi':
        print(x)


35476


In [282]:
# LSI Vector for barbi
barbi_vect = U[35476, :]

In [288]:
# See how all values are nearly zeros here
# This means based on the partial SVD barbi could not be associated with any of the 200 concepts

print(barbi_vect.shape)
print(barbi_vect)

(200,)
[-5.97319730e-05 -7.28706892e-04 -5.56533604e-04 -2.58308827e-04
 -9.04762889e-05  3.58383787e-04  7.01210107e-04  8.65162677e-04
 -1.03141290e-03  2.12860304e-04  9.12255960e-04  4.72981143e-04
  4.26798020e-04  1.67704303e-04 -3.93107137e-04 -6.09517205e-05
 -5.63505667e-04  2.95453529e-04  3.54628678e-04  7.05647361e-05
 -9.20430832e-04 -6.99329150e-04  2.47680013e-04 -1.61162024e-04
  3.16953770e-04 -2.59734097e-04  2.62782550e-04  4.07299315e-05
  1.07873663e-05 -1.87672050e-04 -2.48467102e-04  1.99546400e-05
 -4.05071532e-04  2.54925539e-04  5.57065609e-06  1.90400681e-04
  1.07451298e-04  3.32381537e-04 -2.65230755e-04 -4.96576022e-04
 -4.05292159e-04 -2.47512436e-04 -3.35331468e-04 -4.18421966e-04
 -1.41538632e-05 -3.85928607e-04  3.70511222e-04 -6.56852083e-05
 -4.29612619e-05 -7.51259215e-05 -3.16427400e-04  2.72208779e-04
  2.18086963e-04  1.25468672e-04 -1.09495194e-04  1.61033889e-04
  2.55503573e-04  3.17301076e-04 -7.27439543e-05  5.97535178e-04
 -1.12913099e-04 -

In [289]:
# Find out terms with atleast some values non zero
# This can 'also' be achieved with dot product of the vector with itself 
# Hint: dot product and L2 norm are related

for x in range(total_terms):
    xvect = U[x,:]
    if np.dot(xvect, xvect.T) > 0.1:
        print(x)

0
5
6
10
17
19
22
25
28
34
37
40
41
43
46
48
54
56
61
63
65
67
68
69
70
71
72
74
75
82
96
103
104
106
110
115
118
119
123
132
139
144
145
150
152
153
155
159
160
166
168
170
172
173
176
181
191
192
194
196
199
203
210
211
215
219
221
233
235
236
238
239
242
248
250
251
253
255
257
259
267
269
271
277
278
279
281
288
294
296
298
304
307
308
309
318
321
322
324
332
337
342
344
347
349
357
360
364
365
373
375
380
401
407
412
428
433
444
467
480
482
485
488
491
494
503
504
515
517
520
521
536
538
540
541
547
558
565
577
595
614
622
626
627
643
653
660
683
701
716
721
727
728
736
752
758
759
765
772
813
835
837
840
854
858
863
870
884
890
894
901
911
913
921
940
964
971
989
991
999
1030
1090
1102
1105
1110
1117
1118
1126
1147
1151
1164
1178
1184
1187
1194
1219
1224
1225
1228
1231
1259
1262
1270
1272
1281
1284
1293
1294
1295
1300
1302
1316
1320
1328
1332
1343
1353
1355
1359
1368
1369
1398
1404
1449
1462
1464
1469
1511
1537
1539
1547
1557
1562
1574
1598
1606
1616
1617
1626
1652
1662
1691
1727

In [344]:
# Lets pick up a random word from this list

random_word_id = 1606
print(lex.getLexiconEntry(1606).getKey())

random_word_vector = U[1606, :]
print(random_word_vector)

israel
[-0.00143992 -0.00518304 -0.00858604 -0.01311484 -0.02044043  0.02271794
 -0.0211334   0.00721446 -0.01533262  0.00878013 -0.00904899  0.00308502
  0.0065805   0.00826751  0.00064667  0.00953743 -0.0205393  -0.00944619
  0.02443633  0.00304099  0.00620647  0.00544061 -0.01082816  0.00846017
  0.02518056 -0.03588506  0.00721234  0.02185951  0.005177    0.00502001
  0.01233878 -0.00620205  0.02279748 -0.02528099  0.00374538  0.00147119
  0.01043954  0.00319198  0.01561221  0.01137089 -0.02115125 -0.00437902
  0.03885455  0.01596632 -0.01479378  0.01636691  0.00991835 -0.01513944
 -0.0235409   0.01247361  0.023309   -0.00355279 -0.02150817  0.00494213
 -0.00245587  0.01413194  0.01488091 -0.01981374 -0.03196889 -0.01406634
  0.00489684 -0.00427701 -0.01561316 -0.00267437 -0.01674049 -0.02269596
  0.01492505 -0.02055191  0.01117711 -0.0153797  -0.01009683  0.02057886
 -0.00333316 -0.01458247  0.01450801  0.00696688 -0.04692351 -0.04055942
  0.00691132 -0.04073667 -0.004861    0.0276

In [338]:
# Find similarity with all other words
arr = random_word_vector@U.T

# Sort so that the ids of the top 3 most similar entries are at the top
# Partition sort does not guarantee that the entries apart from these top 3 will be sorted
partitioned_indices = np.argpartition(random_word_vector@U.T, -5)[-5:]

# Get the values and sort them in descending order
top_three_values = arr[partitioned_indices]
sorted_order = np.argsort(top_three_values)[::-1]

# Apply the sorted order to both the values and the indices
top_three_values_sorted = top_three_values[sorted_order]
top_three_indices_sorted = partitioned_indices[sorted_order]

print("Indices: ", top_three_indices_sorted)
print("Values: ",top_three_values_sorted)


Indices:  [1606 1617 1598 1652 3120]
Values:  [0.27389011 0.22343647 0.19992114 0.16656874 0.14712704]


In [339]:
# What words are these?

for id_ in top_three_indices_sorted:
    print(id_, lex.getLexiconEntry(int(id_)).getKey())

1606 israel
1617 palestinian
1598 isra
1652 gaza
3120 hama


'olymp'

In [349]:
# Another random word id 

random_id = 5124 # olymp
random_word_vector = U[5124, :]
partitioned_indices = np.argpartition(random_word_vector@U.T, -5)[-5:]


In [350]:
# Without bothering to sort

for x in partitioned_indices:
    print(lex.getLexiconEntry(int(x)).getKey())

medal
sport
olymp
gold
game
